# Topic Model Evaluation

This notebook contains code needed to run the experiments of my Bachelor's thesis on Topic Modeling Algorithms.

For installation please check out `README.md` of this repository if you haven't already.

In [ ]:
import sys
sys.path.append('../')

In [ ]:
# Import neccessary packages
import numpy as np
from sentence_transformers import SentenceTransformer

from evaluation import DataLoader, Trainer

## Trump's tweets

First we will run BERTopic and Top2Vec on Donald Trump's tweets.

His tweets' archive can be found here: https://www.thetrumparchive.com/

For our experiments we will not consider retweets and deleted tweets, those are being filtered out during the operation of `DataLoader` class.

In [ ]:
dataset, custom = "Trump", True

In [ ]:
%%time
dataloader_trump = DataLoader(dataset=dataset).\
                    prepare_docs(save=f"{dataset}.txt").\
                    preprocess_octis(output_folder=f"{dataset}")

To speed up BERTopic we precalculate embeddings using `SentenceTransformer all-mpnet-base-v2` model. Otherwise, in every different parameter setting would require calculating embeddings, and that would result in a massive runtime.

In [ ]:
# Prepare data
dataset, custom = "Trump", True
dataloader = DataLoader(dataset)
data_trump = dataloader.load_octis(custom)
data_trump = [" ".join(words) for words in data_trump.get_corpus()]

# Extract embeddings
model = SentenceTransformer("all-mpnet-base-v2")
embeddings_trump = model.encode(data_trump, show_progress_bar=True)

In [ ]:
# Save embeddings
np.savetxt('embeddings_trump.txt', embeddings_trump)

In [ ]:
embeddings_trump = np.loadtxt('embeddings_trump.txt')

As described in the Thesis and the BERTopic paper [https://arxiv.org/pdf/2203.05794.pdf] the performance of the topic models is reflected by two widely-used metrics, topic coherence and topic diversity.

For each topic model, coherence is evaluated using Normalized Pointwise Mutual Information (NPMI). This measure ranges from $-1$ to $1$, where $1$ indicates a perfect association.

Topic diversity is the percentage of unique words for all topics. This measure ranges from $0$ to $1$ where $0$ indicates redundant topic and $1$ indicates varied topic.

Ranging from $10$ to $50$ topics with steps of $10$, the NPMI score is calculated at each step for each topic model. Results are averaged across $3$ runs for each step.

In [ ]:
# Evaluating BERTopic on Trump's tweets

for i in range(3):

    params = {
        "embedding_model": "all-mpnet-base-v2",
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 15,
        "verbose": True
    }

    trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings_trump,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"../results/Basic/Trump/bertopic_{i+1}")

In [ ]:
# Evaluating Top2Vec on Trump's tweets

for i in range(3):

    params = {"nr_topics": [(i+1)*10 for i in range(5)],
              "hdbscan_args": {'min_cluster_size': 15,
                               'metric': 'euclidean',
                               'cluster_selection_method': 'eom'}}

    trainer = Trainer(dataset=dataset,
                      custom_dataset=custom,
                      custom_model=None,
                      model_name="Top2Vec",
                      params=params,
                      verbose=True)
    results = trainer.train(save=f"../results/Basic/Trump/Top2Vec_{i+1}")

## 20Newsgroups

The 20Newsgroups dataset comprises 16309 newsgroups posts on 20 topics.

In [ ]:
dataset, custom = "20NewsGroup", False
dataloader_20ng = DataLoader(dataset)
data_20ng = dataloader_20ng.load_octis(custom)
data_20ng = [" ".join(words) for words in data_20ng.get_corpus()]

# Extract embeddings
#model = SentenceTransformer("all-mpnet-base-v2")
#embeddings_20ng = model.encode(data_20ng, show_progress_bar=True)

In [ ]:
# Save embeddings
np.savetxt('embeddings_20ng.txt', embeddings_20ng)

In [ ]:
# Evaluating BERTopic on 20Newsgroups

for i in range(3):

    params = {
        "embedding_model": "all-mpnet-base-v2",
        "nr_topics": [(i+1)*10 for i in range(5)],
        "min_topic_size": 15,
        "verbose": True
    }

    trainer = Trainer(dataset=dataset,
                      model_name="BERTopic",
                      params=params,
                      bt_embeddings=embeddings_20ng,
                      custom_dataset=custom,
                      verbose=True)
    results = trainer.train(save=f"../results/Basic/20NewsGroups/bertopic_{i+1}")

In [ ]:
# Evaluating Top2Vec on 20NewsGroups

for i in range(3):

    params = {"nr_topics": [(i+1)*10 for i in range(5)],
              "hdbscan_args": {'min_cluster_size': 15,
                               'metric': 'euclidean',
                               'cluster_selection_method': 'eom'}}

    trainer = Trainer(dataset=dataset,
                      custom_dataset=custom,
                      custom_model=None,
                      model_name="Top2Vec",
                      params=params,
                      verbose=True)
    results = trainer.train(save=f"../results/Basic/20NewsGroups/Top2Vec_{i+1}")